In [3]:
import json,sys,os,codecs,re
import numpy as np
PHONE_ID=0
SCREEN_SIZE=1
USER_ID=2
TYPING_TIME=3
TYPING_TYPE=4
FINAL_STRING=5
RAW_STRING=6
TYPING_INTERVAL=7
TYPING_TIME_PER_WORD=8
CHOOSING_TIME_PER_WORD=9
TYPING_STATUS=10
BACK_RECORD=11
CORRECT_RECORD=12
TOUCH_INFO=13
PARSE_RESULT=14

kbpos = {}

def readKeyboard():
    with open(r'R9Keyboard.txt','r') as f:
        kbinfo = f.readlines()
    
    for i in kbinfo:
        matchObj = re.search(r'lable :([a-z]) left:(\d+).0 right:(\d+).0 top:(\d+).0 bottom:(\d+).0 center_x:(\d+).0 center_y:(\d+).0',i)
        if matchObj:
            kbpos[matchObj.groups()[0]] = (matchObj.groups()[1],matchObj.groups()[2],matchObj.groups()[3],matchObj.groups()[4],matchObj.groups()[5],matchObj.groups()[6])
        
readKeyboard()

In [5]:
kbpos

{'a': ('66', '163', '510', '357', '114', '434'),
 'b': ('596', '693', '339', '186', '644', '263'),
 'c': ('384', '481', '339', '186', '432', '263'),
 'd': ('278', '375', '510', '357', '326', '434'),
 'e': ('226', '323', '681', '528', '274', '605'),
 'f': ('384', '481', '510', '357', '432', '434'),
 'g': ('490', '587', '510', '357', '538', '434'),
 'h': ('596', '693', '510', '357', '644', '434'),
 'i': ('756', '853', '681', '528', '804', '605'),
 'j': ('702', '799', '510', '357', '750', '434'),
 'k': ('808', '905', '510', '357', '856', '434'),
 'l': ('914', '1011', '510', '357', '962', '434'),
 'm': ('808', '905', '339', '186', '856', '263'),
 'n': ('702', '799', '339', '186', '750', '263'),
 'o': ('862', '959', '681', '528', '910', '605'),
 'p': ('968', '1065', '681', '528', '1016', '605'),
 'q': ('14', '111', '681', '528', '62', '605'),
 'r': ('332', '429', '681', '528', '380', '605'),
 's': ('172', '269', '510', '357', '220', '434'),
 't': ('438', '535', '681', '528', '486', '605'),


In [76]:
import math
'''
typing length
aver typing interval
var of typing interval
aver choosing time
typing status
aver offset
var of offset
'''
dataList = []

kind = {
    u'通讯录':0,
    u'小说阅读':1,
    u'地图':2,
    u'教育学习':3,
    u'搜索':4,
    u'社交网络':5,
    u'游戏':6,
    u'新闻':7,
    u'others':8,
    u'其他':9,
    u'浏览器':10,
    u'网购':11,
    u'手机应用管理':12,
    u'短信':13,
    u'聊天':14,
    u'视频':15,
    u'音乐':16,
    u'NULL':17,
}

def aver(l):
    return np.array(l).mean()


def var(l):
    return np.array(l).var()

def std(l):
    return math.sqrt(var(l))

class infoLine:
    def __init__(self,s):
        self.infobox = s.strip('\n').strip('\r').split('\t')

    def raw_string(self):
        return self.infobox[RAW_STRING]

    def final_string(self):
        return self.infobox[FINAL_STRING]

    def typing_type(self):
        return self.infobox[TYPING_TYPE]
    
    def typing_interval(self):
        return  [int(i) for i in self.infobox[TYPING_INTERVAL].split('#')]

    def typing_time_perword(self):
        return int(self.infobox[TYPING_TIME_PER_WORD])

    def choosing_time_perword(self):
        return int(self.infobox[CHOOSING_TIME_PER_WORD])

    def touch_info(self):
        touch_list = self.infobox[TOUCH_INFO].split('#')[1:-1]
        return  [(int(i.split(',')[0]),int(i.split(',')[1])) for i in touch_list]

    def typing_status(self):
        return self.infobox[TYPING_STATUS]

    def parse_result(self):
        return self.infobox[PARSE_RESULT]

    def has_result(self):
        return len(self.infobox)>=15

    def keyboard_size(self):
        return tuple(map(lambda x: int(x) ,self.infobox[TOUCH_INFO].split('#')[0].split(',')))
    
def dis(t1,t2):
    return math.sqrt((t1[0]-t2[0])**2+(t1[1]-t2[1])**2)

def getData():
    path = r'OPPO-R9m'
    cnt = 0
    for filename in os.listdir(path):
        with codecs.open(path + '\\' + filename,encoding = 'GBK') as f:
            lines = f.readlines()
        
        print filename
        datalist=[]
        for line in lines:
            infoline = infoLine(line)
            if not infoline.has_result():
                continue

            if infoline.typing_type() == 'typing' and infoline.parse_result() in ['AllJP','EndJP','QP'] and len(infoline.raw_string())>1 and infoline.keyboard_size() == (699,0,1080):
                data = []
                
                rs = infoline.raw_string()
                
                if '\'' in rs:
                    continue
                try:
                    data.append(len(rs))
                    data.append(aver(infoline.typing_interval()))
                    data.append(std(infoline.typing_interval()))
                    data.append(kind[infoline.typing_status()])

                    std_pos=[]
                    for i in rs:
                        std_pos.append((int(kbpos[i][4]),int(kbpos[i][5])))
                    offset =  [ dis(std_pos[i],infoline.touch_info()[i]) for i in range(len(infoline.touch_info()))]
                    data.append(aver(offset))
                    data.append(std(offset))

                    if infoline.parse_result() in ['AllJP']:
                        data.append(-1)
                    else:
                        data.append(1)
                    datalist.append(data)
                except:
                    continue

        with codecs.open(r'oppo'+'\\'+filename,'w',encoding = 'GBK') as f:
            f.write(json.dumps(datalist))
        
        print cnt
        cnt += 1
            

In [4]:

import random,json,os
#0 1 2 3 4 5 6
def random_shuffle():
    path = r'oppo/'
    o = 404
    for filename in os.listdir(path)[404:]:
        with open(path+filename,'r') as f:
            data = json.loads(f.read())
        
        random.shuffle(data)
        with open(path+filename,'w') as f:
            f.write(json.dumps(data))
        
        print o
        o += 1
def divide():
    path = r'oppo/'
    o = 0
    for filename in os.listdir(path):
        with open(path+filename,'r') as f:
            data = json.loads(f.read())
        plus_data = []
        minus_data = []
        for i in data:
            if i[6] == 1:
                plus_data.append(i)
            else:
                minus_data.append(i)
        with open(path+filename,'w') as f:
            f.write(json.dumps(plus_data)+'\n'+json.dumps(minus_data))
        
        print o
        o += 1

In [5]:
divide()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [10]:
filename = os.listdir('oppo/')[9]
with open('oppo/'+filename,'r') as f:
    plus_data = json.loads(f.readline())
    minus_data = json.loads(f.readline())
    print len(plus_data),len(minus_data)

71335 3926


In [33]:
def get_train_test_data():
    path = 'oppo/'
    cnt = 0
    for filename in os.listdir(path):
        with open(path+filename,'r') as f:
            plus_data = json.loads(f.readline())
            minus_data = json.loads(f.readline())
        #print len(plus_data),len(minus_data)
        def writeFormat(f,data):
            for line in data:
                f.write(' '.join([str(i) for i in line])+'\n')
        
        with open('train_data/'+filename,'w') as f1:
            writeFormat(f1,plus_data[:1000])
            writeFormat(f1,minus_data[:1000])
        with open('test_data/'+filename,'w') as f1:
            writeFormat(f1,plus_data[1000:])
            writeFormat(f1,minus_data[1000:])
        print cnt,filename
        cnt += 1
        

In [34]:
get_train_test_data()

0 861079036708255+460030879423126
1 861744033735599+460008942838495
2 861744035264036+460002290449294
3 861262036140014+460004343172558
4 861262037912619+460021241638299
5 861262030104834+460025741178363
6 861600034074315+460025671334248
7 861600032968534+460078437156460
8 861262034146617+460037520123794
9 861079034169732+460003602689189
10 862021030379591+460021618545640
11 861744034494790+460075268816007
12 861600039825752+460079289021091
13 861600038612573+460018604016705
14 861744034007550+460000654169605
15 862021030308095+460029163458468
16 861744032024839+460027048470645
17 861262034227995+460031387149424
18 861600032926110+460077252383325
19 861079035056136+460001983969885
20 861079032130314+460023021562721
21 861744039005211+460030368938083
22 861744031925598+460029400631901
23 862021030140837+460017905314344
24 861600031507911+460027641916693
25 861600037220998+460007640396289
26 862021031014759+460036461487750
27 861744038272853+460031308116462
28 861600038339193+46000841292

231 861079037156611+460078646526475
232 861262031904091+460002150647198
233 861600039658591+460010035619158
234 861262030414696+460030261552154
235 861744038803830+460015676513621
236 861744037300812+460027287249224
237 861079031301817+460002257350098
238 861079034009094+460037971021901
239 861262039288430+460030819767647
240 861744031939375+460002674372882
241 861079033450893+460030725719943
242 861744037896215+460011284607494
243 862021032134192+460028799879243
244 861744035213272+460027958936867
245 861744031169718+460005641580572
246 861600036362833+460014315407928
247 861600034556592+460031246487936
248 861744032672652+460036840562037
249 861744033715179+460007354124174
250 862021032114335+460003062770718
251 861262032746319+460005250378133
252 861744030962873+460027055318328
253 861262030347730+460009671837836
254 861744037907715+460013118509975
255 861262037658634+460025560009708
256 861744034155557+460030784291751
257 861600033595732+460078971433336
258 861600039327072+46003799

limit_output extension: Maximum message size of 10000 exceeded with 10006 characters

In [45]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout

def MLP_binary_classifier():
    filename = os.listdir('train_data/')[0]
    train_set = numpy.loadtxt('train_data/'+filename)
    x_train = dataset[:,0:6]
    y_train = dataset[:,6]
    
    test_set = numpy.loadtxt('test_data/'+filename)
    x_test = test_set[:,0:6]
    y_test = test_set[:,6]

    
    model = Sequential()
    model.add(Dense(16, input_dim=6, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(16, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='tanh'))

    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])

    model.fit(x_train, y_train,
              epochs=20,
              batch_size=128)
    score = model.evaluate(x_test, y_test, batch_size=128)
    print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

In [43]:
def justtest():
    filename = os.listdir('train_data/')[0]
    dataset = numpy.loadtxt('train_data/'+filename)
    x_train = dataset[:,0:6]
    y_train = dataset[:,6]
    print x_train